<a href="https://colab.research.google.com/github/opsifiz/AIB5_Emotion-Classification/blob/main/model/test_model_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown https://drive.google.com/uc?id=1KPet67g4PvwEwHoKCo99tR4X1pJtkHrC

Downloading...
From (original): https://drive.google.com/uc?id=1KPet67g4PvwEwHoKCo99tR4X1pJtkHrC
From (redirected): https://drive.google.com/uc?id=1KPet67g4PvwEwHoKCo99tR4X1pJtkHrC&confirm=t&uuid=a52ef86d-99cd-4a23-9043-9e4e8777b5b1
To: /content/my_model01.zip
100% 247M/247M [00:02<00:00, 118MB/s]


In [2]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 9.4 MB/s eta 0:00:00


In [3]:
import zipfile

with zipfile.ZipFile("my_model01.zip", 'r') as zip_ref:
    zip_ref.extractall("my_model")

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import contractions
import html
import re

model_dir = "my_model"  # Path to your unzipped model directory

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir, trust_remote_code=True)


In [5]:
id2label = {
    0: 'Anxiety',
    1: 'BPD',
    2: 'Normal',
    3: 'bipolar',
    4: 'depression',
    5: 'mentalillness',
    6: 'schizophrenia'
}

In [6]:
# Inference function
def predict_status(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(dim=1).item()
    return id2label[predicted_class]

In [7]:
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = contractions.fix(text)
    text = html.unescape(text)
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [8]:
# Example usage
text = input("Let's Analysis Your text! : ")
text = clean_text(text)
diagnosis = predict_status(text)
print(f"Predicted diagnosis: {diagnosis}")

Let's Analysis Your text! : Hello World!
Predicted diagnosis: schizophrenia


In [9]:
import json

msgs = {
  "BPD": [
    "Some days I feel everything, and other days I feel nothing at all.",
    "I wish people understood how intense my emotions get.",
    "It's exhausting feeling like I'm too much and not enough at the same time.",
    "Relationships are hard when you're afraid of abandonment all the time.",
    "I’m trying to trust people, but my brain keeps telling me to push them away.",
    "The emotional rollercoaster never stops, and it’s so draining.",
    "I'm not manipulative; I’m just scared of losing people.",
    "I hate how one comment can ruin my entire day.",
    "Therapy helps, but the progress feels so slow sometimes.",
    "I don’t want to be this way—I’m just trying to survive."
  ],
  "Anxiety": [
    "Overthinking every small detail is exhausting.",
    "Even when things are okay, my brain tells me something bad is coming.",
    "I avoid social situations because I'm afraid of saying the wrong thing.",
    "It's hard to breathe sometimes and I don’t even know why.",
    "I wish I could just relax without feeling guilty.",
    "My mind never stops racing, even when I'm trying to sleep.",
    "I'm constantly worried about things I can't control.",
    "No, I'm not overreacting — this is just how my brain works.",
    "Sometimes I cancel plans just because the anxiety is too much.",
    "I know it's irrational, but it still feels very real."
  ],
  "depression": [
    "Getting out of bed feels like climbing a mountain.",
    "I smile in public, but I'm falling apart inside.",
    "I feel like a burden to everyone around me.",
    "Nothing brings me joy anymore, not even the things I used to love.",
    "I’m tired all the time, no matter how much I sleep.",
    "It’s like I’m living in slow motion while the world speeds past me.",
    "I want to talk about it, but I don’t know how.",
    "It’s not laziness — it’s depression.",
    "I feel empty, even when I'm surrounded by people.",
    "Sometimes I just want everything to stop."
  ],
  "mentalillness": [
    "Living with mental illness is a daily battle.",
    "I wish society treated mental illness like they do physical illness.",
    "There’s nothing weak about asking for help.",
    "Stigma makes it even harder to get support.",
    "I didn’t choose to be this way, but I’m trying to manage it.",
    "Just because you can’t see it doesn’t mean it’s not real.",
    "I’m more than my diagnosis.",
    "Every day is a new challenge, but I’m still here.",
    "Mental illness doesn’t define me, but it affects me.",
    "I'm fighting a war in my mind that no one else can see."
  ],
  "bipolar": [
    "One week I’m on top of the world, the next I can’t get out of bed.",
    "People don’t understand how scary mania can be.",
    "The highs feel amazing, but I know the crash is coming.",
    "Sometimes I miss the mania, even though it wrecked my life.",
    "It’s hard to find balance when your brain keeps swinging.",
    "Medication helps, but it’s not a cure-all.",
    "I feel like two different people depending on the day.",
    "Managing bipolar disorder is like walking a tightrope.",
    "I just want stability, even if that means giving up the highs.",
    "Being productive during a manic phase isn’t worth the crash."
  ],
  "schizophrenia": [
    "Sometimes I can't tell what’s real and what’s not.",
    "The voices don’t define me, but they make life harder.",
    "People fear what they don’t understand.",
    "I’m more than the stigma that surrounds schizophrenia.",
    "Medication helps, but the side effects are tough too.",
    "Paranoia makes it hard to trust even those I love.",
    "I wish people saw the person, not just the illness.",
    "Living with schizophrenia is not the horror story people think it is.",
    "I can have a fulfilling life with the right support.",
    "Every day is a fight for clarity."
  ],
  "Normal": [
    "I feel pretty good today.",
    "Life has its ups and downs, but I'm managing well.",
    "I’m grateful for my health and peace of mind.",
    "Sometimes I'm sad or anxious, but it passes.",
    "I'm not perfect, but I’m content.",
    "I’m doing fine, just living day by day.",
    "Work is stressful sometimes, but I know how to cope.",
    "I enjoy spending time with friends and family.",
    "I feel balanced emotionally most of the time.",
    "Nothing major is bothering me right now."
  ]
}


In [10]:
score = 0
max = 0

for status in msgs:
  cnt = 0
  print(f"Evaluate Model : Status => {status} :")
  for msg in msgs[status]:
    print(f"\tMessage : {msg}")
    res = predict_status(msg)
    print(f"\tPredicted : {res}")
    if(status.lower() == res.lower()):
      print("\t✅")
      cnt += 1
    else:
      print("\t❌")
  print(f"Status : {status} => Score : {cnt}/{len(msgs[status])}")
  score += cnt
  max += len(msgs[status])
  print()

print(f"Total Score : {score}/{max}")

Evaluate Model : Status => BPD :
	Message : Some days I feel everything, and other days I feel nothing at all.
	Predicted : mentalillness
	❌
	Message : I wish people understood how intense my emotions get.
	Predicted : BPD
	✅
	Message : It's exhausting feeling like I'm too much and not enough at the same time.
	Predicted : Anxiety
	❌
	Message : Relationships are hard when you're afraid of abandonment all the time.
	Predicted : BPD
	✅
	Message : I’m trying to trust people, but my brain keeps telling me to push them away.
	Predicted : Anxiety
	❌
	Message : The emotional rollercoaster never stops, and it’s so draining.
	Predicted : BPD
	✅
	Message : I'm not manipulative; I’m just scared of losing people.
	Predicted : BPD
	✅
	Message : I hate how one comment can ruin my entire day.
	Predicted : BPD
	✅
	Message : Therapy helps, but the progress feels so slow sometimes.
	Predicted : schizophrenia
	❌
	Message : I don’t want to be this way—I’m just trying to survive.
	Predicted : depression
	❌